In [1]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

def fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    #coordenadas depots
    while(line.strip()):
        coordinates_depots.append(line.split())    
        line = next(file)
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
            
    line = next(file)
            
        #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
                
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = int(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = int(a1)
    a2 = int(a2)
    b1 = int(b1)
    b2 = int(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [ ]:
file_path = "Inst1"
for filename in os.listdir(file_path):
    print("\nSiguiente archivo\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
                

        for i in range(len(coordinates_customers)):
            for j in range(len(coordinates_customers)):
                b = coordinates_customers[i]
                c = coordinates_customers[j]
                if j==i:
                    continue
                distancias.append(distance_cost(b[0], b[1], c[0], c[1]))
                distancias_c2c.append(distance_cost(b[0], b[1], c[0], c[1]))
                
        #MODELO
        mdl = Model(name="MDRP")
        x = mdl.binary_var_list(len(distancias), name="x")
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(len(distancias))]
        mdl.minimize(mdl.sum(ab))

        #X_dj = yd con yd = 1
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(x[j])
            mdl.add_constraint(sum(xdj) == 1)
            
        #X_jd = yd con yd = 1
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(x[offset])
            mdl.add_constraint(sum(xjd) == 1)

            
        #X_ij = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[n_available_depots*i + j + len(distancias_d2c)])
            for j in range(n_customers-1):
                aux.append(x[(n_customers-1)*i + j + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)
        
        #X_ji = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[j*n_available_depots + i])
            for j in range(n_customers):
                if(j == i):
                    continue
                aux.append(x[j*(n_customers-1) + i + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)

        
        # DL para F-MDRP
        u = mdl.continuous_var_list(n_customers, name="u")
        for i in range(n_customers):
            offset_ij = 99*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = 99*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(u[i] - u[j] + (n_customers-n_available_depots+1)*x[j + offset_ij] + (n_customers-n_available_depots-1)*x[i + offset_ji]  <= n_customers - n_available_depots)
        
        
        # NODE-DL para MDRP
        k = mdl.continuous_var_list(n_customers, name="k")
        
        #Constraint 20
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[j*n_customers + i] <= n_available_depots)
                
        #Constraint 21
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[i*n_available_depots + j + len(distancias_d2c)] <= n_available_depots)
        
        #Constraint 22
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1)-1)*x[j*n_customers + i] <= k[i])
                
        #Constraint 23
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1) - 1)*x[i*n_available_depots + j + len(distancias_d2c)] <= k[i])
        
        #Constraint 24
        for i in range(n_customers):
            offset_ij = 99*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = 99*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(k[i] - k[j] <= (n_available_depots - 1)*(1 - x[j + offset_ij] - x[i + offset_ji]))
        
        mdl.print_information()
        solution = mdl.solve()
        
        # Verificar si se encontró una solución
        if solution:
            # Extraer el valor óptimo de la función objetivo
            print(f"Valor óptimo de la función objetivo: {solution.objective_value}")
        else:
            print("No se encontró solución óptima.")
            
        solution.display()


Siguiente archivo

Número de customers: 100
Número de depots disponibles: 10
a
Model: MDRP
 - number of variables: 12100
   - binary=11900, integer=0, continuous=200
 - number of constraints: 24020
   - linear=24020
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
